In [ ]:
# import os

# # Set your paths
# images_dir = '/content/drive/MyDrive/Colab Notebooks/license_data/images/train'  # change this to your actual path
# labels_dir = '/content/drive/MyDrive/Colab Notebooks/license_data/labels/train'  # change this to your actual path

# # List files
# image_files = set(os.path.splitext(f)[0] for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png')))
# label_files = set(os.path.splitext(f)[0] for f in os.listdir(labels_dir) if f.lower().endswith('.txt'))

# # Files that exist in only one of the sets
# images_without_labels = image_files - label_files
# labels_without_images = label_files - image_files

# # Delete unmatched image files
# for fname in images_without_labels:
#     for ext in ['.jpg', '.jpeg', '.png']:
#         path = os.path.join(images_dir, fname + ext)
#         if os.path.exists(path):
#             os.remove(path)
#             print(f"Deleted image: {path}")

# # Delete unmatched label files
# for fname in labels_without_images:
#     path = os.path.join(labels_dir, fname + '.txt')
#     if os.path.exists(path):
#         os.remove(path)
#         print(f"Deleted label: {path}")


In [ ]:
import importlib.util
if importlib.util.find_spec("ultralytics") is None:
    !pip install ultralytics

!pip install easyocr
!pip install paddleocr paddlepaddle

In [ ]:
from ultralytics import YOLO
import easyocr
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
model=YOLO('yolov8n.pt')

In [ ]:
result= model.train(data="/content/drive/MyDrive/Colab Notebooks/config1.yaml", epochs=25)

In [ ]:
!scp -r /content/runs '/content/drive/MyDrive/Colab Notebooks/license'

In [ ]:
model = YOLO('/content/drive/MyDrive/Colab Notebooks/license/runs/detect/train3/weights/best.pt')

In [ ]:
results = model('/content/drive/MyDrive/Colab Notebooks/download.jpeg', show=True)

In [ ]:
for r in results:
    print(r.boxes)        # Detected boxes
    print(r.names)        # Class names
    print(r.probs)        # Class probabilities

In [ ]:
results[0].save(filename='/content/drive/MyDrive/Colab Notebooks/detected_image.jpg')

In [ ]:
model = YOLO('/content/drive/MyDrive/Colab Notebooks/license/runs/detect/train3/weights/best.pt')

In [ ]:
reader= easyocr.Reader(['en'])

img_path='/content/drive/MyDrive/Colab Notebooks/download.jpeg'
image=cv2.imread(img_path)

In [ ]:
import cv2
import csv
from datetime import datetime
from ultralytics import YOLO
from paddleocr import PaddleOCR

# Load YOLO model (trained for license plate detection)
yolo_model = YOLO("/content/drive/MyDrive/Colab Notebooks/license/runs/detect/train3/weights/best.pt")

# Load PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')

# Image path
img_path = "/content/drive/MyDrive/Colab Notebooks/download.jpeg"
img = cv2.imread(img_path)

# CSV file setup
csv_file = "/content/drive/MyDrive/Colab Notebooks/plate_texts.csv"
with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Timestamp", "Plate_ID", "Detected_Texts", "Avg_Confidence"])  # header


# Function to log OCR results into CSV
def log_ocr_texts_combined(ocr, image, csv_file, plate_id):
    result = ocr.ocr(image)

    if result and isinstance(result, list):
        all_texts = []
        all_scores = []

        for res in result:
            if "rec_texts" in res:
                all_texts.extend(res["rec_texts"])
                all_scores.extend([float(s) for s in res["rec_scores"]])

        if all_texts:
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            combined_text = " | ".join(all_texts)
            avg_confidence = sum(all_scores) / len(all_scores)

            # Print on screen
            print(f"[{timestamp}] PlateID {plate_id} → Texts: {combined_text}, Avg Confidence: {avg_confidence:.2f}")

            # Write to CSV
            with open(csv_file, mode='a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([timestamp, plate_id, combined_text, f"{avg_confidence:.2f}"])


# Detect license plates with YOLO and process with OCR
plate_id = 1
results = yolo_model(img)

for r in results:
    for box in r.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # bounding box
        plate = img[y1:y2, x1:x2]

        log_ocr_texts_combined(ocr, plate, csv_file, plate_id)
        plate_id += 1
